# Implementation of Wav2Vec2.0 algorithm using HuggingFace library, and computation of WER on our Kaggle Dataset

## Install libraries

In [ ]:
pip install huggingsound

In [ ]:
import os
import random
import pandas as pd
import re
import string
from nltk.corpus import stopwords

import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import re
import string
from nltk.corpus import stopwords
from tensorflow.keras.utils import plot_model
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from tensorflow.keras.layers import Embedding, LSTM, add, Concatenate, Reshape, concatenate, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

## Create the Wav2Vec2.0 model

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-french")

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# test on an audio, coming from a radio programme
audio_paths = ["/content/radiomp3_court.mp3"]

transcriptions = model.transcribe(audio_paths)[0]
print(transcriptions["transcription"]) #return the text = transcription, strart_timestamps, end_timestamps, probabilities

## Computing Wav2Vec on our dataset

In [ ]:
#utilsation des données de notre dataset
liste_lesmis = os.listdir("/content/lesmis")
liste_lupin = os.listdir("/content/lupincontresholme")
print(liste_lesmis[:10])

In [ ]:
audio_path = [f'/content/lesmis/{liste_lesmis[0]}']
#print(audio_path)
transcriptions = model.transcribe(audio_path)[0]
print(transcriptions["transcription"])

In [ ]:
N = 300 #take 300 elements of each books to transcript
res = {}
for i in range(N):
    index = random.randint(0, len(liste_lesmis)-1)
    audio_path = [f'/content/lesmis/{liste_lesmis[index]}']
    transcriptions = model.transcribe(audio_path)[0]
    texte = transcriptions["transcription"]
    res[liste_lesmis[index]] = texte

for i in range(N):
    index = random.randint(0, len(liste_lupin)-1)
    audio_path = [f'/content/lupincontresholme/{liste_lupin[index]}']
    transcriptions = model.transcribe(audio_path)[0]
    texte = transcriptions["transcription"]
    res[liste_lupin[index]] = texte

In [ ]:
df = pd.DataFrame(list(res.items()), columns=['filename', 'Texte'])

In [ ]:
#save if needed
df.to_csv('transcriptWavToVeq.csv')
files.download('transcriptWavToVeq.csv')

In [ ]:
df_reel = pd.read_csv('/content/transcript.txt', sep="|", header=None)
df_reel.columns = ["filename", "transcript1", "transcript2", "duration"]

In [ ]:
df_reel = df_reel.iloc[:,:2]
df_reel

In [ ]:
#need to remove punctuation in real transcripts to better make comparisons

punct = list(string.punctuation)
punct.pop(12) #pour conserver les tirets
#df_reel["transcript1"] = df_reel["transcript1"].apply(lambda x: x.replace("'","")) #remove apostrophe et concatenate
for pun in punct :
    df_reel["transcript1"] = df_reel["transcript1"].apply(lambda x: x.replace(pun,"")) #remove ponctuation
df_reel["transcript1"] = df_reel["transcript1"].apply(lambda x: x.lower()) #remove Upercase
df_reel["transcript1"] = df_reel["transcript1"].apply(lambda x: x.replace("\s+"," ")) #remove double space
df_reel.head()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df_W2V = pd.read_csv('/content/transcriptWavToVeq.csv')

In [ ]:
df_W2V

In [ ]:
refs = []
hyps= []
for i in range (df_W2V.shape[0]):
  filename = df_W2V["filename"].iloc[i]
  texte_W2V = df_W2V["Texte"].iloc[i]
  hyps.append(texte_W2V)
  #print(filename)
  #print(texte_W2V)
  lien = filename[:-9]
  filename_reel = lien+"/"+filename
  #print(filename_reel)
  texte_reel = df_reel.loc[df_reel.filename==filename_reel]
  texte_reel = texte_reel.iloc[0, 1]
  refs.append(texte_reel)
  #print(texte_reel)

## Definition of the WER, between the reference and or hypothesis

In [ ]:
def wer(ref, hyp ,debug=True):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#cor " + str(numCor))
        print("#sub " + str(numSub))
        print("#del " + str(numDel))
        print("#ins " + str(numIns))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'numCor':numCor, 'numSub':numSub, 'numIns':numIns, 'numDel':numDel, "numCount": len(r)}

def wers(refs, hyps):
    numSub = 0
    numDel = 0
    numCor = 0
    numIns = 0
    numCount = 0
    for ref, hyp in zip(refs, hyps):
        result = wer(ref, hyp, False)
        numSub += result["numSub"]
        numDel += result["numDel"]
        numCor += result["numCor"]
        numIns += result["numIns"]
        numCount += result["numCount"]
 
    return round( (numSub + numDel + numIns) / (float) (numCount), 3)

In [ ]:
print(wer(texte_reel, texte_W2V ,debug=True))

## Results

In [ ]:
print(wers(refs, hyps))